In [ ]:
from random import random


def sigmoid(value):
    from math import exp
    return 1 / (1 + exp(-value))


class MyLogisticRegressionMultipleLabels:

    def __init__(self):
        self.intercept_ = []
        self.coefficient_ = []

    def fit_batch(self, x, y, learning_rate=0.001, no_epochs=1000):
        self.coefficient_ = []
        self.intercept_ = []
        labels = list(set(y))
        for label in labels:
            coefficient = [random() for _ in range(len(x[0]) + 1)]
            for _ in range(no_epochs):
                errors = [0] * len(coefficient)
                for input, output in zip(x, y):
                    y_computed = sigmoid(self.evaluate(input, coefficient))
                    error = y_computed - 1 if output == label else y_computed
                    for i, xi in enumerate([1] + list(input)):
                        errors[i] += error * xi * y_computed * (1 - y_computed)
                for i in range(len(coefficient)):
                    coefficient[i] = coefficient[i] - learning_rate * errors[i]
            self.intercept_.append(coefficient[0])
            self.coefficient_.append(coefficient[1:])

    def evaluate(self, xi, coefficient):
        yi = coefficient[0]
        for j in range(len(xi)):
            yi += coefficient[j + 1] * xi[j]
        return yi

    def predict_one_sample(self, sample_features):
        predictions = []
        for intercept, coefficient in zip(self.intercept_, self.coefficient_):
            computed_value = self.evaluate(sample_features, [intercept] + coefficient)
            predictions.append(sigmoid(computed_value))
        return predictions.index(max(predictions))

    def predict(self, in_test):
        computed_labels = [self.predict_one_sample(sample) for sample in in_test]
        return computed_labels

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset 
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes) 
input_data = iris.data.features
output_data = iris.data.targets

In [ ]:
input_data_list = [[float(sl), float(sw), float(pl), float(pw)] for sl, sw, pl, pw in
                   zip(input_data['sepal length'], input_data['sepal width'], input_data['petal length'],
                       input_data['petal width'])]

In [ ]:
d = {}
i = 0
for x in output_data['class']:
    if d.get(x, -1) == -1:
        d[x] = i / 3
        i += 1

output_data_list = []
for x in output_data['class']:
    output_data_list.append(d[x])

In [ ]:
def normalize_feature(data_arg):
    minimum = min(data_arg)
    maximum = max(data_arg)
    for i in range(len(data_arg)):
        data_arg[i] = (data_arg[i] - minimum) / (maximum - minimum)


def normalize(train_data_arg):
    new_train_data = [[] for _ in range(len(train_data_arg))]
    for i in range(len(train_data_arg[0])):
        aux_list = [train_data_arg[r][i] for r in range(len(train_data_arg))]
        normalize_feature(aux_list)
        for t in range(len(train_data_arg)):
            new_train_data[t].append(aux_list[t])

    return new_train_data


def normalisation(train_data, test_data):
    if not isinstance(train_data[0], list):
        train_data = [[d] for d in train_data]
        test_data = [[d] for d in test_data]

        normalised_train_data = normalize(train_data)
        normalised_test_data = normalize(test_data)

        normalised_train_data = [d[0] for d in normalised_train_data]
        normalised_test_data = [d[0] for d in normalised_test_data]
    else:
        normalised_train_data = normalize(train_data)
        normalised_test_data = normalize(test_data)

    return normalised_train_data, normalised_test_data

In [ ]:
from random import shuffle

indexes = [i for i in range(len(input_data_list))]
shuffle(indexes)
train_indexes = indexes[:int(0.75 * len(input_data_list))]
train_input_data = [input_data_list[i] for i in range(len(input_data_list)) if i in train_indexes]
test_input_data = [input_data_list[i] for i in range(len(input_data_list)) if i not in train_indexes]
train_output_data = [output_data_list[i] for i in range(len(input_data_list)) if i in train_indexes]
test_output_data = [output_data_list[i] for i in range(len(input_data_list)) if i not in train_indexes]

train_input_data, test_input_data = normalisation(train_input_data, test_input_data)

model = MyLogisticRegressionMultipleLabels()

model.fit_batch(train_input_data, train_output_data)

predicted = model.predict(test_input_data)
print(predicted)

In [ ]:
err = 0
for y, r in zip(predicted, test_output_data):
    err += (y - r) ** 2

print(f"Error {err / len(test_output_data)}")

In [ ]:
a = [[5.35, 3.85, 1.25, 0.4]]
result = model.predict(a)
if result[0] == 0:
    print("setosa")
elif result[0] == 1:
    print("versicolor")
else:
    print("virginica")